In [80]:
import requests
from bs4 import BeautifulSoup
import re

In [191]:
def get_full_data(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    titel = soup.select_one("h1.grid-title").get_text(strip=True)
    ecli = soup.find("div", class_="rol-metadata-blok").find(string=re.compile("ECLI:")).get_text(strip=True)
    datum = soup.find(string=re.compile("Datum uitspraak")).find_next('dd').contents[0].get_text(strip=True)
    samenvatting = soup.find(string=re.compile("Inhoudsindicatie")).find_next('dd').contents[0].get_text(strip=True)
    
    trefwoorden_element = soup.find("ul", class_="trefwoorden").find_all("li")
    trefwoorden = []
    for trefwoord in trefwoorden_element:
        title = trefwoord.attrs["title"]
        if title.startswith("Proceduresoort "):
            trefwoorden.append(("proceduresoort", title.split("Proceduresoort ")[1]))
        elif title.startswith("Rechtsgebied "):
            trefwoorden.append(("rechtsgebied", title.split("Rechtsgebied ")[1]))
    
    inhoud = soup.find("div", id="volledigetekst").find("div", class_="iprox-rich-content").get_text()
    
    return titel, ecli, datum, samenvatting, trefwoorden, inhoud

In [184]:
url = "https://www.raadvanstate.nl/uitspraken/?zoeken=true&zoeken_term=&pager_rows=10&kalenderjaar=2023&actualiteit=&Zoe_Selected_facet%3ARechtsgebied=57"

In [185]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}
response = requests.get(url, headers=headers)

In [186]:
soup = BeautifulSoup(response.content, "html.parser")

In [192]:
uitspraken = []
for element in soup.select("div.rol-entry"):
    link = element.select_one("div.grid-title a.siteLink")["href"]
    titel, ecli, datum, samenvatting, trefwoorden, inhoud = get_full_data(link, headers)
    uitspraken.append((link, titel, ecli, datum, samenvatting, trefwoorden, inhoud))

In [193]:
len(uitspraken)

10

In [194]:
uitspraken[0]

('https://www.raadvanstate.nl/uitspraken/@136072/202100531-1-r3/',
 'Uitspraak 202100531/1/R3',
 'ECLI:NL:RVS:2023:932',
 '8 maart 2023',
 'Bij besluit van 8 december 2020 heeft het college van burgemeester en wethouders van Midden-Drenthe het wijzigingsplan "Buitengebied Midden-Drenthe, [locatie 1] te Garminge" vastgesteld. Het wijzigingsplan voorziet in een wijziging van het bestemmingsplan "De Broeksteeg", vastgesteld door de raad van de gemeente Midden-Drenthe op 28 maart 2013, om een uitbreiding van het op het perceel [locatie 1] in Garminge gevestigde agrarisch bedrijf van [partij] mogelijk te maken. Het bedrijf is een akkerbouwbedrijf dat in hoofdzaak aardappelen teelt, rooit, sorteert en opslaat en bestaat uit enkele gebouwen voor sorteren en opslaan van aardappelen. [appellant] woont op het perceel [locatie 2] in Garminge en kan zich niet verenigen met het wijzigingsplan. Zij vreest onder meer voor geluidsoverlast bij haar woning, als gevolg van de uitbreiding van het agrarisc

In [17]:
elements = soup.select("div.rol-entry")

In [18]:
len(elements)

448

In [21]:
link = elements[0].select_one("div.grid-title a.siteLink")["href"]

In [22]:
uitspraak = requests.get(link, headers=headers)

In [24]:
uitspraak = BeautifulSoup(uitspraak.content, "html.parser")

In [28]:
uitspraak.select_one("div.rol-metadata-blok dt:contains('ECLI') + dd").get_text(strip=True)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [41]:
uitspraak.select_one("div.rol-metadata-blok dt:contains('ECLI')", recursive=False)

<dd>ECLI:NL:RVS:2023:932
                        <dt>Datum uitspraak
                        <dd>8 maart 2023
                        <dt>Inhoudsindicatie
                        <dd>Bij besluit van 8 december 2020 heeft het college van burgemeester en wethouders van Midden-Drenthe het wijzigingsplan "Buitengebied Midden-Drenthe, [locatie 1] te Garminge" vastgesteld. Het wijzigingsplan voorziet in een wijziging van het bestemmingsplan "De Broeksteeg", vastgesteld door de raad van de gemeente Midden-Drenthe op 28 maart 2013, om een uitbreiding van het op het perceel [locatie 1] in Garminge gevestigde agrarisch bedrijf van [partij] mogelijk te maken. Het bedrijf is een akkerbouwbedrijf dat in hoofdzaak aardappelen teelt, rooit, sorteert en opslaat en bestaat uit enkele gebouwen voor sorteren en opslaan van aardappelen. [appellant] woont op het perceel [locatie 2] in Garminge en kan zich niet verenigen met het wijzigingsplan. Zij vreest onder meer voor geluidsoverlast bij haar woning, als

In [107]:
blok = uitspraak.find("div", class_="rol-metadata-blok")

In [176]:
inhoud = uitspraak.find("div", id="volledigetekst").find("div", class_="iprox-rich-content").get_text()

In [177]:
print(inhoud)


202100531/1/R3. Datum uitspraak: 8 maart 2023
AFDELING BESTUURSRECHTSPRAAK
Uitspraak in het geding tussen:
[appellant], wonend te Garminge, gemeente Midden-Drenthe,
en
het college van burgemeester en wethouders van Midden-Drenthe,
verweerder.
Procesverloop
Bij besluit van 8 december 2020 heeft het college het wijzigingsplan "Buitengebied Midden-Drenthe, [locatie 1] te Garminge" vastgesteld.
Tegen dit besluit heeft [appellant] beroep ingesteld.
Bij besluit van 6 april 2021 heeft het college het wijzigingsplan "Buitengebied Midden-Drenthe, [locatie 1] te Garminge" opnieuw en gewijzigd vastgesteld.
Bij besluit van 14 september 2021 heeft het college het wijzigingsplan opnieuw en gewijzigd vastgesteld onder de naam "De Broekstreek, [locatie 1] te Garminge".
Het college heeft een verweerschrift ingediend.
[appellant] en het college hebben nadere stukken ingediend.
De Afdeling heeft de zaak op een zitting behandeld op 15 september 2022, waar [appellant] en het college, vertegenwoordigd door

In [162]:
for child in ecli:
    print(child.contents[0].get_text(strip=True))
    print(child.attrs["title"])

Eerste aanleg - meervoudig
Proceduresoort Eerste aanleg - meervoudig
RO - Drenthe
Rechtsgebied RO - Drenthe


In [71]:
for child in ecli.children:
    print(child)

Datum uitspraak
                        
<dd>8 maart 2023
                        <dt>Inhoudsindicatie
                        <dd>Bij besluit van 8 december 2020 heeft het college van burgemeester en wethouders van Midden-Drenthe het wijzigingsplan "Buitengebied Midden-Drenthe, [locatie 1] te Garminge" vastgesteld. Het wijzigingsplan voorziet in een wijziging van het bestemmingsplan "De Broeksteeg", vastgesteld door de raad van de gemeente Midden-Drenthe op 28 maart 2013, om een uitbreiding van het op het perceel [locatie 1] in Garminge gevestigde agrarisch bedrijf van [partij] mogelijk te maken. Het bedrijf is een akkerbouwbedrijf dat in hoofdzaak aardappelen teelt, rooit, sorteert en opslaat en bestaat uit enkele gebouwen voor sorteren en opslaan van aardappelen. [appellant] woont op het perceel [locatie 2] in Garminge en kan zich niet verenigen met het wijzigingsplan. Zij vreest onder meer voor geluidsoverlast bij haar woning, als gevolg van de uitbreiding van het agrarisch bedrijf

In [59]:
uitspraak

<!DOCTYPE html>

<html lang="nl">
<head prefix="og: http://ogp.me/ns# dcterms: http://purl.org/dc/terms/ overheid: http://standaarden.overheid.nl/owms/">
<meta charset="utf-8"/>
<meta content="initial-scale=1" name="viewport"/>
<title>Uitspraak 202100531/1/R3 - Raad van State</title>
<link href="https://www.raadvanstate.nl/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="https://www.raadvanstate.nl/favicon.png" rel="icon"/><!--[if IE]>
            <link rel="shortcut icon" href="https://www.raadvanstate.nl/favicon.ico" />
          <![endif]-->
<meta content="#ffffff" name="msapplication-TileColor"/>
<meta content="/publish/varianten/1/favicons/256x256_shortcut-icon-256_1.png" name="msapplication-TileImage"/>
<link href="https://www.raadvanstate.nl/@136072/202100531-1-r3/" rel="canonical"/><!--
      ****************************************************************************************
      Deze website draait op het IPROX-platform van InfoProjects bv.
      IPROX is een s

In [1]:
from datetime import datetime
import locale

def parse_dutch_datetime(date_string):
    # Set the locale to Dutch
    locale.setlocale(locale.LC_TIME, 'nl_NL')

    try:
        parsed_datetime = datetime.strptime(date_string, '%d %B %Y')
        return parsed_datetime
    except ValueError:
        return None

# Usage
dutch_date_string = "30 mei 2023"
parsed_datetime = parse_dutch_datetime(dutch_date_string)

In [3]:
parsed_datetime

datetime.datetime(2023, 5, 30, 0, 0)